In [1]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
pd.options.display.float_format = '{:,.3f}'.format

pd.options.mode.chained_assignment = None
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")
#sns.set(rc={'figure.figsize':(10,10)})
import matplotlib
#matplotlib.use('agg')
#matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
#plt.rcParams.update(plt.rcParamsDefault)
new_plot_col=list(range(1900,2001,10))
from scipy.stats.stats import pearsonr

In [2]:
new_plot_col

[1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000]

In [3]:
def lemma_maker(x, y):
    return lemmatizer.lemmatize(x,y)
br_to_us=pd.read_excel("Book.xlsx",skiprows=[0])
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
spelling_replacement={'modifier':br_to_us_dict,'head':br_to_us_dict}

In [4]:
contextwords_df=pd.read_csv("contexts.csv",sep="\t")
contextwords=contextwords_df.context.tolist()

In [5]:
reddy11_study=pd.read_csv("/data/dharp/compounding/datasets/ijcnlp_compositionality_data/MeanAndDeviations.clean.txt",sep="\t")
#print(reddy11_study.columns)
reddy11_study.columns=['compound','to_divide']
reddy11_study['modifier_mean'],reddy11_study['modifier_std'],reddy11_study['head_mean'],reddy11_study['head_std'],reddy11_study['compound_mean'],reddy11_study['compound_std'],_=reddy11_study.to_divide.str.split(" ",7).str
reddy11_study['modifier'],reddy11_study['head']=reddy11_study['compound'].str.split(" ",2).str
reddy11_study.modifier=reddy11_study.modifier.str[:-2]
reddy11_study['head']=reddy11_study['head'].str[:-2]
reddy11_study.drop(['compound','to_divide'],axis=1,inplace=True)
#reddy11_study['modifier']=np.vectorize(lemma_maker)(reddy11_study['modifier'],'n')
reddy11_study['head']=np.vectorize(lemma_maker)(reddy11_study['head'],'n')
reddy11_study.replace(spelling_replacement,inplace=True)
reddy11_study['modifier']=reddy11_study['modifier']+"_n"
reddy11_study['head']=reddy11_study['head']+"_n"
reddy11_study=reddy11_study.apply(pd.to_numeric, errors='ignore')
#reddy11_study.set_index(['modifier','head'],inplace=True)
reddy11_study.head()

,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
0,3.867,1.118,4.867,0.340,4.250,0.871,end_n,user_n
1,1.607,1.655,1.893,1.496,1.704,1.717,firing_n,line_n
2,2.821,1.965,4.862,0.345,3.828,1.234,game_n,plan_n
3,4.767,0.423,4.862,0.345,4.800,0.476,application_n,form_n
4,0.600,0.800,4.586,1.099,1.310,1.021,snail_n,mail_n


In [14]:
compounds=pd.read_csv("/data/dharp/compounding/datasets/phrases.csv")
compounds.columns=['modifier','head','context','decade','count']
#compounds=compounds.query('decade != 2000')
#compounds['joiner']=compounds['modifier']+" "+compounds['head']
#compounds=compounds.loc[compounds.modifier.isin(contextwords) & (compounds['head'].isin(contextwords))]
compounds=compounds.groupby(['modifier','head','decade','context'])['count'].sum().to_frame()
display(compounds.shape)
compounds.head()

(67609736, 1)

count
modifier head decade context       
----_n   x_n  1920   x_n          3
              1930   x_n          6
              1940   x_n         36
              1950   x_n         24
              1960   x_n         57

In [20]:
compounds.describe()

,count
count,"67,609,736.000"
mean,65.606
std,"1,552.464"
min,1.000
25%,4.000
50%,10.000
75%,30.000
max,"2,969,254.000"


In [15]:
compounds_new=compounds[compounds['count']>20]
print((compounds.shape[0]-compounds_new.shape[0])/compounds.shape[0])
display(compounds_new.shape)
compounds_new.head()

0.6714089077348269


(22215957, 1)

count
modifier head decade context       
----_n   x_n  1940   x_n         36
              1950   x_n         24
              1960   x_n         57
              1970   x_n         96
              1980   x_n         66

## Before

In [18]:
count_df=reddy11_study[['modifier','head']].merge(compounds.reset_index(),on=['modifier','head'],how='left')
#temp_df.set_index(['modifier','head','decade'],inplace=True)
count_df=pd.pivot_table(count_df,index=['modifier','head'],columns='decade',values='count',aggfunc=np.sum)
to_change=[str(x).split('.')[0] for x in count_df.columns.tolist()]
count_df.columns=to_change
count_df.columns = count_df.columns.map(int)
count_df.fillna(0,inplace=True)

In [19]:
selected_df=count_df[new_plot_col]
display(selected_df[(selected_df != 0).all(1)].shape)
selected_df[(selected_df != 0).all(1)]

(48, 11)

,,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000
modifier,head,,,,,,,,,,,
acid_n,test_n,31.000,40.000,25.000,11.000,21.000,29.000,68.000,"1,083.000","1,111.000","1,833.000","2,509.000"
application_n,form_n,262.000,478.000,571.000,822.000,"2,044.000","4,139.000","7,212.000","13,419.000","22,021.000","40,863.000","45,599.000"
balance_n,sheet_n,"20,098.000","44,050.000","79,364.000","77,583.000","79,401.000","96,872.000","139,725.000","177,386.000","295,322.000","473,133.000","692,443.000"
bank_n,account_n,"2,827.000","4,790.000","4,351.000","5,480.000","5,738.000","9,323.000","16,037.000","26,371.000","44,295.000","84,441.000","154,829.000"
brass_n,ring_n,303.000,251.000,173.000,132.000,149.000,152.000,247.000,416.000,778.000,"1,581.000","2,647.000"
brick_n,wall_n,"8,679.000","6,770.000","5,575.000","5,114.000","4,979.000","7,099.000","10,218.000","13,300.000","19,562.000","33,060.000","60,296.000"
case_n,study_n,59.000,264.000,"2,509.000","10,194.000","11,906.000","30,714.000","81,011.000","221,248.000","400,248.000","756,828.000","1,288,775.000"
chain_n,reaction_n,26.000,69.000,78.000,657.000,"7,116.000","22,379.000","35,653.000","36,036.000","44,007.000","160,381.000","210,176.000"
china_n,clay_n,577.000,328.000,376.000,276.000,312.000,385.000,418.000,586.000,533.000,429.000,604.000


In [9]:
compound_counts[compound_counts['count']>20].head(20)

count
modifier head      decade       
a_n      addison_n 1960       28
                   1970       96
                   1980      206
                   1990      196
                   2000      222
         anti_n    1960       27
                   1970       30
                   1980       41
                   1990       67
                   2000       90
         archive_n 1960       98
                   1970      258
                   1980       91
                   1990       60
                   2000       29
         atom_n    1950       29
                   1960       34
                   2000       25
         b_n       1810       58
                   1820       74

In [ ]:
compound_counts[compound_counts['count']>20]

In [14]:
compounds.reset_index(inplace=True)
new_compounds=compounds.loc[compounds.modifier.isin(contextwords) & (compounds['head'].isin(contextwords))]
new_compounds_decade_counts=new_compounds.groupby(['modifier','head'])['count'].sum().to_frame()
new_compounds_decade_counts

count
modifier head                 
a_n      addison_n         526
         anti_n            169
         archive_n         510
         atom_n            116
         b_n             24600
         ballet_n           85
         band_n            384
         baseball_n         90
         behavior_n       1080
         bell_n            665
         belle_n            78
         bend_n            304
         berkley_n         144
         bibliography_n   2799
         bill_n            336
         bird_n            215
         board_n            80
         book_n             37
         boswell_n         214
         brand_n            48
         bunch_n           514
         burn_n            104
         business_n         70
         c_n               756
         calendar_n         78
         call_n             39
         campaign_n        240
         candle_n           48
         cassell_n         510
         cat_n             388
...                        ...
zurich_n ch_n               54
         chamber_n         283
         church_n           28
         city_n             53
         cohort_n          169
         conference_n      300
         congress_n         47
         dada_n            102
         federal_n          42
         government_n      245
         institute_n        98
         james_n           146
         market_n           53
         medical_n         211
         opera_n            44
         peasantry_n       141
         polytechnic_n     363
         school_n          432
         seminar_n        1253
         speech_n           43
         stock_n           431
         translation_n      39
         university_n      371
         wilhelm_n          48
zvi_n    bodie_n            97
         institute_n       918
         press_n            28
         publication_n      38
zygote_n nucleus_n          42
         result_n           72

[848435 rows x 1 columns]

In [15]:
new_compounds_decade_counts.describe()

,count
count,848435.000
mean,2673.265
std,148101.622
min,1.000
25%,74.000
50%,184.000
75%,554.000
max,118319029.000
